In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [40]:
fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = 450,
    mode = "gauge+number",
    title = {'text': "Speed"},
    # delta = {'reference': 380},
    gauge = {'axis': {'range': [-100, 600]},
             'bar': {'color': "white"},
             'steps' : [
                 {'range': [-100, 0], 'color': "#ea5b0c"},
                 {'range': [0, 400], 'color': "#f3d193"},
                 {'range': [400, 600], 'color': "#00ae9f"}],
             'threshold' : {'line': {'color': "white", 'width': 4}, 'thickness': 1, 'value': 450}}))

fig.update_layout(paper_bgcolor = "lavender", font = {'color': "grey", 'family': "Arial"})
fig.show()


In [ ]:
def draw_guage(title: str, start:float, end:float, absolute_sep:float, industry_average: float, value: float) -> go.Figure:
    # start is industry minimum
    # end is industry maximum
    # absolute sep is likely going to be 0 (there may not be an absolute separator)
    # industry average is going to be average for firms in the same sector
    # value is the ratio for the firm
    pass

